In [1]:
%load_ext autoreload
%autoreload 2
from benchmark.utils import *
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
from benchmark.construct_threatrace_graph import *
from benchmark.threatrace import *
#from benchmark import sql_api
import logging 
from torch_geometric import seed_everything
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')

2023-07-05 08:26:02


In [2]:
import configparser
config_dir = "/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs"
config_file_record_03 = config_dir + '/eng3/cadets/cadets_03_record.ini'
config_rec3 = configparser.ConfigParser()
config_rec3.read(config_file_record_03)


['/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs/eng3/cadets/cadets_03_record.ini']

In [3]:
def print_data_stats(data):
    print()
    print(data)
    print('===========================================================================================================')

    # Gather some statistics about the graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Number of training nodes: {data.train_mask.sum()}')
    print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

In [4]:
#print type of config_rec3
print(type(config_rec3))

<class 'configparser.ConfigParser'>


In [5]:
# #tt gt
# # read txt file row by row and fill list with values
# file_path = "/Users/robinbuchta/Documents/GitHub/threaTrace/groundtruth/cadets.txt"
# def read_txt_file(file_path):
#     with open(file_path, 'r') as file:
#         data = file.read().splitlines()
#     return data
# tt_gt_uuids = read_txt_file(file_path)
# print(len(tt_gt_uuids))
# tt_mapped_atk_nodes,translation = find_mapping_value_for_gt_uuid(config_rec3, tt_gt_uuids)
# tt_mapped_atk_nodes_decremented = [x - 1 for x in tt_mapped_atk_nodes]

tt_mapped_atk_nodes_decremented = get_atk_nodes_for_evaluation(config_rec3)


File exists.


SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('','7180A298-3E80-11E8-A5CB-3FA3753A265A','A654C900-3E80-11E8-A5CB-3FA3753A265A','1896F4CB-3F57-11E8-A5CB-3FA3753A265A','4735DB1F-3E41-11E8-A5CB-3FA3753A265A','2E80BBAC-0872-5BEA-B37C-DD37384874A7','9A755703-3E80-11E8-A5CB-3FA3753A265A','A2EBF1B4-5CC6-535A-BB14-9E027E9ADAF4','46B2E0FF-3F44-11E8-A5CB-3FA3753A265A','AF07E019-DAB1-595A-84B5-E4D554161702','89A6B7A6-3E80-11E8-A5CB-3FA3753A265A','790B0B4A-3E80-11E8-A5CB-3FA3753A265A','A059DDD0-3E80-11E8-A5CB-3FA3753A265A','D6E9B9E7-3EAC-11E8-A5CB-3FA3753A265A','2B23AEB2-3E36-11E8-A5CB-3FA3753A265A','5B9ACAD5-1F43-574C-8E52-11C9F59E1DAD','D393B770-3E98-11E8-A5CB-3FA3753A265A','C037B4FE-B6AB-5313-BAEC-1BB512A5A5C2','FA271928-3F2C-11E8-A5CB-3FA3753A265A','9AACB930-3E80-11E8-A5CB-3FA3753A265A','651A1485-3F0F-11E8-A5CB-3FA3753A265A','20A6D0BC-3EA2-11E8-A5CB-3FA3753A265A','806C99C5-3E80-11E8-A5CB-3FA3753A265A','9BBA20DB-3E80-11E8-A5CB-3FA3753A265A','87934FFD-3E80-11E8-A5CB-3FA3753A265A',

In [6]:
tt_data_train = create_tt_orig_data_train()
tt_data_test, test_adj, test_adj2 = create_tt_orig_data_test()

In [7]:
tt_data_train.num_nodes

362645

In [8]:
tt_data_test.num_nodes

357174

In [9]:
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent=15, extension="15percentTrain") # 4min
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print(data_train)
print(data_test)

Data(x=[125787, 56], edge_index=[2, 1426653], y=[125787], train_mask=[125787], test_mask=[125787])
Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])


In [10]:
data_test.y.unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([133448,  43080, 374521,  12657,  10510,    122]))

In [11]:
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent=40, extension="40percentTrain") # 4min
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print(data_train)
print(data_test)

Data(x=[244917, 56], edge_index=[2, 2900550], y=[244917], train_mask=[244917], test_mask=[244917])
Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])


In [12]:
print_data_stats(tt_data_train) # Number of nodes: 950774 Number of edges: 12192401 # 10% =  Number of nodes: 127452 Number of edges: 1261612
print_data_stats(tt_data_test) # Number of nodes: 942812 Number of edges: 12082603
tt_data_test.n_id = torch.arange(tt_data_test.num_nodes)
tt_data_train.n_id = torch.arange(tt_data_train.num_nodes)


Data(x=[362645, 56], edge_index=[2, 4347080], y=[362645], train_mask=[362645], test_mask=[362645], n_id=[362645])
Number of nodes: 362645
Number of edges: 4347080
Average node degree: 11.99
Number of training nodes: 362645
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False

Data(x=[357174, 56], edge_index=[2, 4316489], y=[357174], train_mask=[357174], test_mask=[357174], n_id=[357174])
Number of nodes: 357174
Number of edges: 4316489
Average node degree: 12.09
Number of training nodes: 357174
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False


In [13]:
data_train

Data(x=[244917, 56], edge_index=[2, 2900550], y=[244917], train_mask=[244917], test_mask=[244917])

In [14]:
data_test

Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])

In [15]:
# data_train.coalesce()
# data_test.coalesce()

In [16]:
data_train

Data(x=[244917, 56], edge_index=[2, 2900550], y=[244917], train_mask=[244917], test_mask=[244917])

In [17]:
super(type(ttp.train_model))

NameError: name 'ttp' is not defined

In [19]:
ttp = ThreaTracePipeline(config_rec3, data_train, data_test)

In [20]:
type(ttp.train_loader)


torch_geometric.loader.neighbor_loader.NeighborLoader

In [21]:
ttp.delete_old_models()
ttp.pretraining()
ttp.multi_model_training()

1 17.23710823059082 0.1080039360273072 2023-07-05 08:26:56
2 9.110090255737305 0.10862047142501337 2023-07-05 08:26:57
3 31.042341232299805 0.10667695586668137 2023-07-05 08:26:58
4 13.272482872009277 0.1686285557964535 2023-07-05 08:26:59
5 28.637380599975586 0.16900827627318643 2023-07-05 08:27:00
6 17.23926544189453 0.18920287280997236 2023-07-05 08:27:02
7 21.098596572875977 0.19522532123127426 2023-07-05 08:27:03
8 14.725915908813477 0.20111303012857418 2023-07-05 08:27:04
9 15.66061019897461 0.20375474140218933 2023-07-05 08:27:05
10 17.039621353149414 0.20517971394390752 2023-07-05 08:27:06
11 12.38233470916748 0.7496498813883887 2023-07-05 08:27:07
12 12.021464347839355 0.7528591318691639 2023-07-05 08:27:08
13 9.232730865478516 0.7538513047277241 2023-07-05 08:27:10
14 16.17337989807129 0.7553579375870193 2023-07-05 08:27:11
15 6.147055149078369 0.7564930160013392 2023-07-05 08:27:12
16 13.503554344177246 0.7705957528468829 2023-07-05 08:27:13
17 12.27341365814209 0.7770714160

In [30]:
atk_nodes = get_atk_nodes(config_rec3)
mapped_atk_nodes, translation = find_mapping_value_for_gt_uuid(config_rec3, atk_nodes)
print(len(mapped_atk_nodes))

mapped_atk_nodes_decremented = [x - 1 for x in mapped_atk_nodes]

2023-07-05 14:31:43
File exists.
SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('','7180A298-3E80-11E8-A5CB-3FA3753A265A','A654C900-3E80-11E8-A5CB-3FA3753A265A','1896F4CB-3F57-11E8-A5CB-3FA3753A265A','4735DB1F-3E41-11E8-A5CB-3FA3753A265A','2E80BBAC-0872-5BEA-B37C-DD37384874A7','9A755703-3E80-11E8-A5CB-3FA3753A265A','A2EBF1B4-5CC6-535A-BB14-9E027E9ADAF4','46B2E0FF-3F44-11E8-A5CB-3FA3753A265A','AF07E019-DAB1-595A-84B5-E4D554161702','89A6B7A6-3E80-11E8-A5CB-3FA3753A265A','790B0B4A-3E80-11E8-A5CB-3FA3753A265A','A059DDD0-3E80-11E8-A5CB-3FA3753A265A','D6E9B9E7-3EAC-11E8-A5CB-3FA3753A265A','2B23AEB2-3E36-11E8-A5CB-3FA3753A265A','5B9ACAD5-1F43-574C-8E52-11C9F59E1DAD','D393B770-3E98-11E8-A5CB-3FA3753A265A','C037B4FE-B6AB-5313-BAEC-1BB512A5A5C2','FA271928-3F2C-11E8-A5CB-3FA3753A265A','9AACB930-3E80-11E8-A5CB-3FA3753A265A','651A1485-3F0F-11E8-A5CB-3FA3753A265A','20A6D0BC-3EA2-11E8-A5CB-3FA3753A265A','806C99C5-3E80-11E8-A5CB-3FA3753A265A','9BBA20DB-3E80-11E8-A5CB-3FA3753A265A','87934

In [31]:
type(atk_nodes)

set

In [25]:
gt = tt_mapped_atk_nodes_decremented
print("##############################################################################################################")
print("own evaluation")
print("##############################################################################################################")
ttp.reinit_test_data()
ttp.test_model_performance()
print("##############################################################################################################")
print("2 HOP")
hits = ttp.evaluation(gt)


#TP: 12846, FP: 0, TN: 561481, FN: 11
#TP: 12793, FP: 15, TN: 561466, FN: 64

# TP: 18283, FP: 0, TN: 549883, FN: 6172
# Precision:  0.9999999999999947
# Recall:  0.7476180740134911
# F-Score:  0.8555851934556483
# Accuracy:  0.9892537147115462

# TP: 4331, FP: 0, TN: 549883, FN: 20124 #coalense
# Precision:  0.9999999999999769
# Recall:  0.17710079738294757
# F-Score:  0.30091016463781234
# Accuracy:  0.9649613990368041

# TP: 17123, FP: 0, TN: 549883, FN: 7332 # no coalense
# Precision:  0.9999999999999942
# Recall:  0.7001840114495985
# F-Score:  0.823656741497561
# Accuracy:  0.9872339980986806

##############################################################################################################
own evaluation
##############################################################################################################
Loop_num: 0  Accuracy:0.7411  true_classified:425639  false_classified:148699
Loop_num: 1  Accuracy:0.8377  true_classified:481123  false_classified:93215
Loop_num: 2  Accuracy:0.0298  true_classified:17132  false_classified:557206
Loop_num: 3  Accuracy:0.0266  true_classified:15275  false_classified:559063
Loop_num: 4  Accuracy:0.0245  true_classified:14087  false_classified:560251
Loop_num: 5  Accuracy:0.0375  true_classified:21544  false_classified:552794
Loop_num: 6  Accuracy:0.0552  true_classified:31704  false_classified:542634
Loop_num: 7  Accuracy:0.0283  true_classified:16256  false_classified:558082
Loop_num: 8  Accuracy:0.7051  true_classified:404979  false_classified:169359
Loop_num: 9  Accuracy:0.0319  true_classified:18299  false_classifie

100%|██████████| 33/33 [00:00<00:00, 53.62it/s]


{'fn': 20375, 'tn': 549883, 'tp': 4080}
Intersection Counter: 24
574338
4080 0 549883 20375
TP: 4080, FP: 0, TN: 549883, FN: 20375
Precision:  0.9999999999999755
Recall:  0.16683704763851906
F-Score:  0.2859646048467011
Accuracy:  0.9645243741490203


In [26]:
x = ttp.get_detection_insights()

2023-07-05 08:39:04
Get Detection Insights
Unique Count of data_flow.y:  (tensor([0, 1, 2, 3, 4, 5]), tensor([133448,  43080, 374521,  12657,  10510,    122]))
Unique Count of pred: (tensor([False,  True]), tensor([574305,     33]))
Unique Count of filtered_tensor: (tensor([0, 1, 2]), tensor([ 9, 22,  2]))


In [27]:
#tt gt
# read txt file row by row and fill list with values
file_path = "/Users/robinbuchta/Documents/GitHub/threaTrace/groundtruth/cadets.txt"
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()
    return data
tt_gt_uuids = read_txt_file(file_path)
print(len(tt_gt_uuids))
tt_mapped_atk_nodes,translation = find_mapping_value_for_gt_uuid(config_rec3, tt_gt_uuids)
tt_mapped_atk_nodes_decremented = [x - 1 for x in tt_mapped_atk_nodes]

gt = tt_mapped_atk_nodes_decremented
print("##############################################################################################################")
print("own evaluation")
print("##############################################################################################################")
ttp.reinit_test_data()
ttp.test_model_performance()
print("##############################################################################################################")
print("2 HOP")
hits = ttp.evaluation(gt)


#TP: 12846, FP: 0, TN: 561481, FN: 11 
#TP: 12793, FP: 15, TN: 561466, FN: 64 # 40%

#TP: 18204, FP: 0, TN: 549883, FN: 6251


#TP: 18283, FP: 0, TN: 549883, FN: 6172
# Precision:  0.9999999999999947
# Recall:  0.7476180740134911
# F-Score:  0.8555851934556483
# Accuracy:  0.9892537147115462

#TP: 18, FP: 0, TN: 561481, FN: 12839 #coalense

# TP: 12809, FP: 0, TN: 561481, FN: 48 # no coalense
# Precision:  0.9999999999999922
# Recall:  0.9962666251847165
# F-Score:  0.998129821503799
# Accuracy:  0.9999164255194676

12858
SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('9EF37E2E-3E80-11E8-A5CB-3FA3753A265A','A4DD7C60-3E80-11E8-A5CB-3FA3753A265A','A55A32F5-3E80-11E8-A5CB-3FA3753A265A','878A0030-3E80-11E8-A5CB-3FA3753A265A','8145E23B-3E80-11E8-A5CB-3FA3753A265A','A54E7444-3E80-11E8-A5CB-3FA3753A265A','A69A88DD-3E80-11E8-A5CB-3FA3753A265A','80265456-3E80-11E8-A5CB-3FA3753A265A','7172780D-3E80-11E8-A5CB-3FA3753A265A','875F1C7B-3E80-11E8-A5CB-3FA3753A265A','797A71AF-3E80-11E8-A5CB-3FA3753A265A','89268999-3E80-11E8-A5CB-3FA3753A265A','89CE2920-3E80-11E8-A5CB-3FA3753A265A','9F1FABF0-3E80-11E8-A5CB-3FA3753A265A','A547982E-3E80-11E8-A5CB-3FA3753A265A','792914DC-3E80-11E8-A5CB-3FA3753A265A','80629A3A-3E80-11E8-A5CB-3FA3753A265A','89D227A2-3E80-11E8-A5CB-3FA3753A265A','A6654946-3E80-11E8-A5CB-3FA3753A265A','7A72825A-3E80-11E8-A5CB-3FA3753A265A','9EDFF511-3E80-11E8-A5CB-3FA3753A265A','71E1962F-3E80-11E8-A5CB-3FA3753A265A','876EFAB3-3E80-11E8-A5CB-3FA3753A265A','9EECA0F5-3E80-11E8-A5CB-3FA3753A265

100%|██████████| 33/33 [00:00<00:00, 56.47it/s]


{'fn': 12844, 'tn': 561481, 'tp': 13}
Intersection Counter: 22
574338
13 0 561481 12844
TP: 13, FP: 0, TN: 561481, FN: 12844
Precision:  0.9999999999923077
Recall:  0.0010111223458038345
F-Score:  0.002020202020000173
Accuracy:  0.9776368619175468


In [29]:
type(gt)

list

In [28]:
x = ttp.get_detection_insights()

2023-07-05 08:46:31
Get Detection Insights
Unique Count of data_flow.y:  (tensor([0, 1, 2, 3, 4, 5]), tensor([133448,  43080, 374521,  12657,  10510,    122]))
Unique Count of pred: (tensor([False,  True]), tensor([574305,     33]))
Unique Count of filtered_tensor: (tensor([0, 1, 2]), tensor([ 9, 22,  2]))
